In [ ]:
# HTML에서 태그 선택하기
# CSS selector
# 선택자 > 자식
# 선택자 자손
# .클래스 , #ID
# 태그[속성1='속성값1']
# 선택되는게 한개인지, 여러개인지, 없는지 구별 잘하기

In [ ]:
# 학습 목표 : Beautifulsoup 사용해서 웹 사이트의 원하는 데이터를 수집한다.
# requests : 웹 사이트로부터 데이터(html, json 등)를 얻는 lib
# Beautifulsoup는 html을 parsing해주는 lib

import requests

response = requests.get("https://www.naver.com")
print(type(response))
print("status: ", response.status_code)
print("header: ", response.headers)
print("---------------------")
print("html : ", response.text)

In [ ]:
import requests
# https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC
params={"pageNo" : 1, "rangType" : "ALL", "orderBy" : "sim", "keyword" : "뉴진스"}
url = "https://section.blog.naver.com/Search/Post.naver"
response = requests.get(url, params=params) # 파라미터를 넘길 수 있다.
# requests.post() 포스트방식은 url로 파라미터 전달 못함
print(type(response))
print("status: ", response.status_code)
print("header: ", response.headers)
print("---------------------")
print("html : ", response.text)

In [ ]:
# 네이버 블로그에서 얻은 html을 해석해서, 제목만 출력해보자
import requests
from bs4 import BeautifulSoup as bs

params={"pageNo" : 1, "rangType" : "ALL", "orderBy" : "sim", "keyword" : "도쿄여행"}
url="https://section.blog.naver.com/BlogHome.naver?directoryNo=0&currentPage=1&groupId=0"
response = requests.get(url, params=params) # 파라미터를 넘길 수 있다.
html=bs(response.text, 'html.parser') 
# Beautifulsoup에선 html을 변환뿐만 아니라 xml 등도 parsing 해줍니다. xhtml.parser . . .
# html.select_one()
# html.select() # CSS selector를 통해 원하는 태그를 얻을 수 있습니다.
html.select("#content > section > div.area_list_search > div:nth-child(1) > div > div.info_post > div.desc > a.desc_inner")

In [ ]:
#네이버 삼성전자 검색 - 뉴스 탭 - 제목만 출력해보기
import requests
from bs4 import BeautifulSoup as bs

url="https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90"
response=requests.get(url)
html=bs(response.text, 'html.parser')
li_list=html.select(".list_news li")
for li in li_list:
    aTag = li.select_one("a[title]")
    print(aTag.text)
    print()



In [ ]:
#네이버 삼성전자 검색 - 뉴스 탭 - 제목만 출력해보기
import requests
from bs4 import BeautifulSoup as bs

url="https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90"
response=requests.get(url)
html=bs(response.text, 'html.parser')
response = html.select("div > div.news_contents > a[title]")
print(response)


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

url="https://en.wikipedia.org/wiki/List_of_most-visited_websites"
response=requests.get(url)
html=bs(response.text, 'html.parser')
table=html.select_one("table")
tbody=table.select_one("tbody")
tr_list=table.select("tr")

rank_df=pd.DataFrame(columns=("Website","DomainName","Ranking","Type","Company","Country"))

for tr in tr_list[2:] :
    website=tr.select_one("th").text
    td_list = tr.select("td")
    domain_name=td_list[0].text
    ranking=td_list[1].text    #1,2,3,4,5,6
    type=td_list[3].text
    company=td_list[4].text
    country=td_list[5].text
    rank_df.loc[ranking] = [website,domain_name,ranking,type,company,country]

print(rank_df) 

In [67]:
import requests
from bs4 import BeautifulSoup as bs

url="https://www.melon.com/chart/index.htm" #멜론 뮤직차트
# rank,title,artists만 dataFrame에 담아보세요
#    rank     title    artists
# 31  31       eta      뉴진스
# 2   2        제목       가수
response=requests.get(url)
html=bs(response.text, 'html.parser')
tr_list = html.select("#frm > div > table > tbody")
print(tr_list)

[]


In [98]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

url = "https://www.melon.com/chart/index.htm"
# rank,title,artists만 dataFrame에 담아보세요
#    rank     title    artists
# 31  31       eta      뉴진스
# 2   2        제목       가수
response=requests.get(url)
response = requests.get(url)
html = bs(response.text, 'html.parser')

# CSS selector 수정
tr_list = html.select("#frm > div > table > tbody > tr")

# 빈 리스트 초기화
rank_list = []
title_list = []
artists_list = []

for tr in tr_list:
    rank = tr.select_one(".rank").text
    title = tr.select_one(".ellipsis.rank01").text
    artists = tr.select_one(".ellipsis.rank02").text
    
    rank_list.append(rank)
    title_list.append(title)
    artists_list.append(artists)

# 데이터프레임 생성
df = pd.DataFrame({
    'rank': rank_list,
    'title': title_list,
    'artists': artists_list
})

print(df)

Empty DataFrame
Columns: [rank, title, artists]
Index: []


In [100]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

url = "https://music.bugs.co.kr/chart"

response=requests.get(url)
response = requests.get(url)
html = bs(response.text, 'html.parser')
rank_list = []
title_list = []
artists_list = []


# tr_list selector 수정
tr_list = html.select("#CHARTrealtime > table > tbody > tr")
# print(tr_list)


for li in tr_list:
    rank = li.select_one(".ranking > strong").text.strip()
    title = li.select_one(".title").text.strip()
    artists = li.select_one(".artist").text.strip()
    
    rank_list.append(rank)
    title_list.append(title)
    artists_list.append(artists)


# 데이터프레임 생성
df = pd.DataFrame({
    'rank': rank_list,
    'title': title_list,
    'artists': artists_list
})


print(df)

   rank                title            artists
0     1             Magnetic         아일릿(ILLIT)
1     2      나는 아픈 건 딱 질색이니까            (여자)아이들
2     3     첫 만남은 계획대로 되지 않아          TWS (투어스)
3     4  Welcome to the Show        DAY6 (데이식스)
4     5                  밤양갱           비비(BIBI)
..  ...                  ...                ...
95   96  혹시 세상에 혼자 남겨진 것 같다면          우디(Woody)
96   97       Off The Record          IVE (아이브)
97   98       Bye My Monster          온앤오프(ONF)
98   99              Fallin'                홍이삭
99  100                 BODY  하이라이트 (HIGHLIGHT)

[100 rows x 3 columns]


In [101]:
# 선생님 버전
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

url = "https://music.bugs.co.kr/chart"
response=requests.get(url)
html=bs(response.text, 'html.parser')
table=html.select_one(".byChart")
tr_list=table.select("tr")
rank_df=pd.DataFrame(columns=("rank","title","artist"))
for tr in tr_list[1:] :
    rank=tr.select_one("strong").text
    title=tr.select("a[title]")[0].text
    artist=tr.select("a[title]")[1].text
    rank_df.loc[rank]=[rank,title,artist]

print(rank_df)

    rank                title             artist
1      1             Magnetic         아일릿(ILLIT)
2      2      나는 아픈 건 딱 질색이니까            (여자)아이들
3      3     첫 만남은 계획대로 되지 않아          TWS (투어스)
4      4  Welcome to the Show        DAY6 (데이식스)
5      5                  밤양갱           비비(BIBI)
..   ...                  ...                ...
96    96  혹시 세상에 혼자 남겨진 것 같다면          우디(Woody)
97    97       Off The Record          IVE (아이브)
98    98       Bye My Monster          온앤오프(ONF)
99    99              Fallin'                홍이삭
100  100                 BODY  하이라이트 (HIGHLIGHT)

[100 rows x 3 columns]


In [3]:
# 네이버 증권도 됨
# 네이버 시리즈온도 됨
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

url = "https://finance.naver.com/sise/"
response=requests.get(url)
html=bs(response.text, 'html.parser')
# a_list = html.select(".lst_pop li > a")
# print(a_list)

ul=html.select_one(".lst_pop")
b_list = ul.select("li")
finance_df=pd.DataFrame(columns=("회사명","전일","시가","고가","저가","거래량","거래대금"))
index = 1

for items in b_list :
    a=items.select_one("a")
    # print(a)
    sub_url="https://finance.naver.com" + a.attrs["href"]
    # print(sub_url)
    sub_response=requests.get(sub_url)
    sub_html=bs(sub_response.text,'html.parser')
    # print(sub_html)
    
    # 똑같이 여러분들이 select, select_one 써서 원하는 데이터 얻으시면 됩니다.
    # 전일,시가,고가,저가, 거래량, 거래대금 을 pd로 넣으면 될거고 인덱스는 이름으로
    companyName=sub_html.select_one(".blind dd:nth-of-type(2)").text[4::] # 이름
    
    table = sub_html.select(".blind")
    # print(table)
    # 전일가
    yesterday = sub_html.select_one(".blind dd:nth-of-type(5)").text[4::]
    
    # 시가
    time = sub_html.select_one(".blind dd:nth-of-type(6)").text[3::]
    # 고가
    high = sub_html.select_one(".blind dd:nth-of-type(7)").text[3::]
    # 저가
    low = sub_html.select_one(".blind dd:nth-of-type(9)").text[3::]
    # 거래량
    plus = sub_html.select_one(".blind dd:nth-of-type(11)").text[4::]
    # 거래대금
    price = sub_html.select_one(".blind dd:nth-of-type(12)").text[5:-2]
    
    finance_df.loc[index]=[companyName,yesterday,time,high,low,plus,price]
    # print(finance_df)
    index += 1
finance_df

,회사명,전일,시가,고가,저가,거래량,거래대금
1,삼성전자,"83,700","82,900","83,200","81,200","26,110,522","2,142,373"
2,SK하이닉스,"187,400","185,400","188,400","183,000","2,635,889","489,741"
3,한미반도체,"145,500","144,900","148,500","142,500","2,168,498","315,839"
4,HLB,"93,000","92,000","99,900","90,300","2,468,980","237,423"
5,HPSP,"42,950","42,200","44,150","41,850","1,661,496","72,029"
6,엔켐,"361,500","347,000","350,000","316,000","980,849","325,937"
7,디아이,"18,940","18,950","23,150","18,750","14,256,870","309,153"
8,POSCO홀딩스,"395,000","387,000","392,000","386,000","386,100","150,186"
9,카카오,"47,850","47,100","48,000","46,850","797,650","37,805"
10,에코프로비엠,"228,500","222,500","233,500","222,000","529,167","120,886"
